In [95]:
import pandas as pd 
import ast 

In [96]:
movies_df = pd.read_csv("/content/movies_metadata.csv").dropna()
credits_df = pd.read_csv("/content/credits.csv").dropna()
keywords_df = pd.read_csv("/content/keywords.csv").dropna()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
68,False,"{'id': 43563, 'name': 'Friday Collection', 'po...",3500000,"[{'id': 35, 'name': 'Comedy'}]",http://www.newline.com/properties/friday.html,10634,tt0113118,en,Friday,Craig and Smokey are two guys in Los Angeles h...,14.5696,/kkoGT4bnK5oGiNbzGGcXzrXMSxn.jpg,"[{'name': 'New Line Cinema', 'id': 12}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-04-26,28215918.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A lot can go down between thursday and saturda...,Friday,False,7.0,513.0
69,False,"{'id': 10924, 'name': 'From Dusk Till Dawn Col...",19000000,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",http://www.miramax.com/movie/from-dusk-till-dawn/,755,tt0116367,en,From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,15.3392,/ce0d4kM5KxT4x1Oq8JkSuNYevri.jpg,"[{'name': 'A Band Apart', 'id': 59}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-01-19,25836616.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One night is all that stands between them and ...,From Dusk Till Dawn,False,6.9,1644.0
153,False,"{'id': 439053, 'name': 'Brooklyn Cigar Store C...",2000000,"[{'id': 35, 'name': 'Comedy'}]",http://miramax.com/movie/blue-in-the-face/,5894,tt0112541,en,Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",11.5281,/qa9yf3OlfXbYBb0af0mpawGB6TI.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-09-15,1275000.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Welcome to the planet Brooklyn.,Blue in the Face,False,6.8,28.0
178,False,"{'id': 286162, 'name': 'Power Rangers Collecti...",15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.powerrangers.com/,9070,tt0113820,en,Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,7.02423,/A3ijhraMN0tvpDnPoyVP7NulkSr.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1995-06-30,66000000.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Power Is On!,Mighty Morphin Power Rangers: The Movie,False,5.2,153.0


In [98]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [99]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [100]:
movies_df = movies_df.drop(columns=['adult','belongs_to_collection','homepage', 'imdb_id',
                       'overview','popularity','poster_path', 
                       'runtime','spoken_languages','status',
                       'tagline', 'vote_average','vote_count'])
movies_df.head()

,budget,genres,id,original_language,original_title,production_companies,production_countries,release_date,revenue,title,video
9,58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",710,en,GoldenEye,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034.0,GoldenEye,False
68,3500000,"[{'id': 35, 'name': 'Comedy'}]",10634,en,Friday,"[{'name': 'New Line Cinema', 'id': 12}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-04-26,28215918.0,Friday,False
69,19000000,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",755,en,From Dusk Till Dawn,"[{'name': 'A Band Apart', 'id': 59}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-01-19,25836616.0,From Dusk Till Dawn,False
153,2000000,"[{'id': 35, 'name': 'Comedy'}]",5894,en,Blue in the Face,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-09-15,1275000.0,Blue in the Face,False
178,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9070,en,Mighty Morphin Power Rangers: The Movie,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1995-06-30,66000000.0,Mighty Morphin Power Rangers: The Movie,False


In [101]:
def resolve_jsons(df, column_name, ):
    i = 0
    for col in df[column_name]:
        new_col = []
        try:
            col = ast.literal_eval(col)
            for c in col:
                new_col.append(c["name"])
            df[column_name][i] = new_col
        except (ValueError,TypeError):
            pass
        i+=1
    

In [102]:
resolve_jsons(movies_df, 'genres' )
resolve_jsons(movies_df, 'production_companies')
resolve_jsons(movies_df, 'production_countries')
resolve_jsons(keywords_df, 'keywords')
resolve_jsons(credits_df,'cast')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [103]:
i = 0
for col in credits_df["crew"]:
    new_col = []
    try:
            col = ast.literal_eval(col)
            for c in col:
                if c["job"] == "Director":
                    new_col.append(c["name"])
            credits_df["crew"][i] = new_col
    except (ValueError,TypeError):
        pass
    i+=1
credits_df = credits_df.rename(columns={"crew":"director"})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [104]:
credits_df.head()


,cast,director,id
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],11862


In [105]:
 credits_df = credits_df.merge(keywords_df,how='right',on='id')

In [106]:
credits_df.head()

,cast,director,id,keywords
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],862,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],8844,"[board game, disappearance, based on children'..."
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],15602,"[fishing, best friend, duringcreditsstinger, o..."
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],31357,"[based on novel, interracial relationship, sin..."
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],11862,"[baby, midlife crisis, confidence, aging, daug..."


In [107]:
movies_df['id']=movies_df['id'].astype(int).astype("int64")

movies_df = movies_df.merge(credits_df)

In [108]:
movies_df.head(10)

,budget,genres,id,original_language,original_title,production_companies,production_countries,release_date,revenue,title,video,cast,director,keywords
0,58000000,"[Adventure, Science Fiction]",710,en,GoldenEye,"[Universal Pictures, Amblin Entertainment]",[United States of America],1995-11-16,352194034.0,GoldenEye,False,"[Pierce Brosnan, Sean Bean, Izabella Scorupco,...",[Martin Campbell],"[cuba, falsely accused, secret identity, compu..."
1,3500000,"[Adventure, Action, Thriller]",10634,en,Friday,"[United Artists, Eon Productions]",[United Kingdom],1995-04-26,28215918.0,Friday,False,"[Ice Cube, Chris Tucker, Nia Long, Tom Lister ...",[F. Gary Gray],"[rap music, parent child relationship, rapper,..."
2,19000000,"[Adventure, Action, Thriller]",755,en,From Dusk Till Dawn,"[United Artists, Eon Productions]","[United Kingdom, United States of America]",1996-01-19,25836616.0,From Dusk Till Dawn,False,"[George Clooney, Quentin Tarantino, Harvey Kei...",[Robert Rodriguez],"[dancing, brother brother relationship, sexual..."
3,2000000,"[Comedy, Documentary]",5894,en,Blue in the Face,[],[United States of America],1995-09-15,1275000.0,Blue in the Face,False,"[Harvey Keitel, Lou Reed, Michael J. Fox, Rose...","[Paul Auster, Wayne Wang]","[smoking, corner shop, cigarette, tobacco, cig..."
4,15000000,"[Horror, Thriller]",9070,en,Mighty Morphin Power Rangers: The Movie,"[Fox Searchlight Pictures, Dune Entertainment,...","[France, United States of America]",1995-06-30,66000000.0,Mighty Morphin Power Rangers: The Movie,False,"[Amy Jo Johnson, Jason David Frank, David Yost...",[Bryan Spicer],"[based on tv series, tokusatsu, superhero team..."
5,27000,"[Horror, Action, Thriller]",2292,en,Clerks,[Dimension Films],[United States of America],1994-09-13,3151130.0,Clerks,False,"[Brian O'Halloran, Jeff Anderson, Jason Mewes,...",[Kevin Smith],"[salesclerk, loser, aftercreditsstinger]"
6,11000000,"[Comedy, Drama, Family, Music, TV Movie]",11,en,Star Wars,"[Disney Channel, Alan Sacks Productions]",[United States of America],1977-05-25,775398007.0,Star Wars,False,"[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",[George Lucas],"[android, galaxy, hermit, death star, lightsab..."
7,35000000,"[Fantasy, Adventure, Animation, Family]",22586,en,The Swan Princess,"[DreamWorks Animation, Vertigo Entertainment, ...",[United States of America],1994-11-18,9771658.0,The Swan Princess,False,"[Michelle Nicastro, Jack Palance, Howard McGil...",[Richard Rich],"[princess, swan]"
8,45000000,"[Family, Comedy]",8587,en,The Lion King,"[Fox 2000 Pictures, Color Force]",[United States of America],1994-06-23,788241776.0,The Lion King,False,"[Jonathan Taylor Thomas, Matthew Broderick, Ja...","[Roger Allers, Rob Minkoff]","[loss of parents, wild boar, uncle, shaman, re..."
9,63000000,"[Action, Drama, Horror, Science Fiction, Thril...",329,en,Jurassic Park,"[Paramount Pictures, GK Films, Skydance Produc...","[United States of America, Malta]",1993-06-11,920100000.0,Jurassic Park,False,"[Sam Neill, Laura Dern, Jeff Goldblum, Richard...",[Steven Spielberg],"[exotic island, dna, paleontology, tyrannosaur..."
